In [21]:
import re
from collections import defaultdict
import string
import requests
import json
from scipy.special import softmax
import numpy as np
from Models_inf import answer_clues, setup_closedbook
from Strict_json import json_CA_json_converter
from Crossword_inf import Crossword
import datetime
from tqdm import tqdm

In [9]:
def getGrid(dateStr):
    headers = {
        'Referer': 'https://www.xwordinfo.com/JSON/'
    }
    # mm/dd/yyyy
    url = 'https://www.xwordinfo.com/JSON/Data.ashx?date=' + dateStr
    response = requests.get(url, headers=headers)
    context = {}
    grid_data = {}
    if response.status_code == 200:
        bytevalue = response.content
        jsonText = bytevalue.decode('utf-8').replace("'", '"')
        grid_data = json.loads(jsonText)
        return grid_data
    else:
        print(f"Request failed with status code {response.status_code}.")

In [14]:
def extract_data_from_puz(puzzle):
    puzzle = json_CA_json_converter(puzzle, False)
    for dim in ['across', 'down']:
        for grid_num in puzzle['clues'][dim].keys():
            clue_answer_list = puzzle['clues'][dim][grid_num]
            clue_section = clue_answer_list[0]
            ans_section = clue_answer_list[1]
            clue_section = clue_section.replace("&quot;", "").replace("&#39;", "").replace("<em>", "").replace("</em>", "")
            puzzle['clues'][dim][grid_num] = [clue_section, ans_section]
            
    all_clues = puzzle['clues']
    across_clue_data = []
    down_clue_data = []
    
    for dim in ['across', 'down']:
        for key in all_clues[dim].keys():
            clue = all_clues[dim][key][0]
            if dim == 'across':
                across_clue_data.append([key, clue])
            else:
                down_clue_data.append([key, clue])
            
    all_clue_info = [across_clue_data, down_clue_data]
    crossword = Crossword(puzzle)
    all_pairs = []
    for var in crossword.variables:
        all_pairs.append((crossword.variables[var]['clue'], crossword.variables[var]['gold']))
    return all_pairs

In [24]:
start_date = datetime.date(2023, 7, 1)
end_date = datetime.date(2023, 7, 30)
overall_data = []
current_date = start_date

for i in tqdm(range((end_date - start_date).days), ncols = 100):
    formatted_date = current_date.strftime('%m/%d/%Y')
    try:
        puzzle = getGrid(formatted_date)
        overall_data.extend(extract_data_from_puz(puzzle))
    except:
        print("Problem with fetching data!!!")
    current_date += datetime.timedelta(days = 1)

 79%|█████████████████████████████████████████████████▉             | 23/29 [00:24<00:06,  1.09s/it]

Problem with fetching data!!!


100%|███████████████████████████████████████████████████████████████| 29/29 [00:31<00:00,  1.08s/it]


In [30]:
all_clues = [clue for clue, _ in overall_data]

In [32]:
MODEL_PATH = "./Inference_components/distilbert_EPOCHs_7_COMPLETE.bin", 
ANS_TSV_PATH = "./Inference_components/all_answer_list.tsv",
DENSE_EMBD_PATH = "./Inference_components/distilbert_7_epochs_embeddings.pkl"
MODEL_TYPE = "distilbert"

In [ ]:
dpr = setup_closedbook(MODEL_PATH, ANS_TSV_PATH, DENSE_EMBD_PATH, 0, MODEL_TYPE)
all_words, all_scores = answer_clues(dpr, all_clues, max_answers = 40000, output_strings=True) 

Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = distilbert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_distilbert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 32


In [ ]:
count = 0
for i, answer_found_list in enumerate(all_words):
    correct_answer = all_answers[i]
    if correct_answer in answer_found_list[:1000]:
        count += 1
print(count / len(all_words))